In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, concat, col, desc, year, month, asc, count, avg, countDistinct
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as func

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import  MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler,Normalizer,StandardScaler,IDF,StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# from matplotlib import pyplot as plt
from datetime import date
from functools import reduce
import numpy as np

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1583729079315_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# create a Spark session
spark = SparkSession.builder.appName("sparkify").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 加载和清洗数据
在这个 Workspace 中，小数据集的名称是 `mini_sparkify_event_data.json`.加载和清洗数据集，检查是否有无效或缺失数据——例如，没有userid或sessionid的数据。 

In [3]:
data = spark.read.json("s3n://udacity-dsnd/sparkify/sparkify_event_data.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# check detail for data
print(data.count())
print(data.describe())
print(data.printSchema())
print(data.show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

26259199
DataFrame[summary: string, artist: string, auth: string, firstName: string, gender: string, itemInSession: string, lastName: string, length: string, level: string, location: string, method: string, page: string, registration: string, sessionId: string, song: string, status: string, ts: string, userAgent: string, userId: string]
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- 

In [5]:
data = data.dropna(how = "any",subset=["userId","sessionId"])
data = data.filter(data["userId"] != "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 探索性数据分析
当你使用完整数据集时，通过加载小数据集，在 Spark 中完成基础操作来实现探索性数据分析。在这个 Workspace 中，我们已经提供给你一个你可以探索的小数据集。

### 定义客户流失

在你完成初步分析之后，创建一列 `Churn` 作为模型的标签。我建议你使用 `Cancellation Confirmation` 事件来定义客户流失，该事件在付费或免费客户身上都有发生。作为一个奖励任务，你也可以深入了解 `Downgrade` 事件。

### 探索数据
你定义好客户流失后，就可以执行一些探索性数据分析，观察留存用户和流失用户的行为。你可以首先把这两类用户的数据聚合到一起，观察固定时间内某个特定动作出现的次数或者播放音乐的数量。

In [6]:
churn_func = udf(lambda x: 1 if x == "Cancellation Confirmation" or x == "Downgrade" else 0, IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
data = data.withColumn("Churn", churn_func(data.page))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# length visual
# 由于AWS的PySpark空间没有pandas，所以无法执行下面的可视化代码
# data_pd = data.select("length").toPandas()
# data_pd.plot(kind = "hist", bins = 500)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
get_datetime_func = udf(lambda x : date.fromtimestamp(x/1000).isoformat())
data = data.withColumn("dt", get_datetime_func(data.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
group_dataset = data.groupBy(["userId","dt","Churn","page"]) \
    .agg({"page":"count"}) \
    .withColumnRenamed("count(page)","cp") \
    .orderBy([desc("userId"),desc("dt")])

# group_dataset.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 特征工程
熟悉了数据之后，就可以构建你认为会对训练模型帮助最大的特征。要处理完整数据集，你可以按照下述步骤：
- 写一个脚本来从小数据集中提取你需要的特征
- 确保你的脚本可以拓展到大数据集上，使用之前教过的最佳实践原则
- 在完整数据集上运行你的脚本，按运行情况调试代码

如果是在教室的 workspace，你可以直接用里面提供的小数据集来提取特征。确保当你开始使用 Spark 集群的时候，把上述的成果迁移到大数据集上。

### Numberic Features

In [11]:
# count songs
df_songs = data.groupBy("userId").agg(countDistinct("song").alias("countSong")).orderBy("userId")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# calc avg listen time
df_avg_length = data.groupBy("userId").agg(avg("length").alias("avgLength")).orderBy("userId")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# count all artist for each user
df_singers = data.dropDuplicates(["userId", "artist"]).groupBy("userId").agg(count("artist").alias("countArtist")).orderBy("userId")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Category Features

In [14]:
# select category features
df_catgory = data.select(["userId", "gender", "level", "location", "method"])
df_catgory = df_catgory.dropDuplicates(["userId"]).orderBy(desc("userId"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#  user may have 2 Churn values, we keep value 1 only
df_calced = data.select("Churn","userId").orderBy(desc("Churn")).dropDuplicates(["userId"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# join all features
for feature in [df_songs, df_avg_length, df_singers, df_catgory]:
    df_calced = df_calced.join(feature, ["userId"], how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# drop all na value
df_calced = df_calced.na.drop()
df_calced = df_calced.dropna(how = "any")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# convert category features to numberic
for index in ["gender", "level", "location", "method"]:
    indexer = StringIndexer(inputCol=index, outputCol=f"{index}Indexer")
    _fit = indexer.fit(df_calced)
    df_calced = _fit.transform(df_calced)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
import pyspark.sql.functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# df_agg = df_calced.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df_calced.columns])
# df_agg.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# convert all features to vector
assembler = VectorAssembler(inputCols=["countSong", "avgLength", "countArtist", "genderIndexer", "levelIndexer", "locationIndexer", "methodIndexer"], outputCol="featuresVec")
df_calced = assembler.transform(df_calced)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# standarded scaler the features column
stander = StandardScaler(inputCol="featuresVec", outputCol="features")
stander_fit = stander.fit(df_calced)
df_calced = stander_fit.transform(df_calced)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# drop useless column
drop_columns = ["userId", "gender", "level", "location", "method"]
for column in drop_columns:
    df_calced = df_calced.drop(column)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# 建模
将完整数据集分成训练集、测试集和验证集。测试几种你学过的机器学习方法。评价不同机器学习方法的准确率，根据情况调节参数。根据准确率你挑选出表现最好的那个模型，然后报告在训练集上的结果。因为流失顾客数据集很小，我建议选用 F1 score 作为优化指标。

In [24]:
# use 70% data as train dataset, and 15% for test and val dataset
train_dataset, test_dataset, val_dataset = df_calced.randomSplit([0.7, 0.15, 0.15], seed = 58)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### LinearRegression

In [25]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="Churn")
binary_evaluator = BinaryClassificationEvaluator(labelCol="Churn", rawPredictionCol="prediction")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# build linear regression model
lr = LinearRegression(labelCol="Churn", featuresCol="features", fitIntercept=False, regParam=0.0, solver="normal")
model = lr.fit(train_dataset) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# train the dataset by using linear regression model
train_pre = model.transform(train_dataset)
test_pre = model.transform(test_dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
print("LinearRegression areaUnderPR for train dataset:", binary_evaluator.evaluate(train_pre, {binary_evaluator.metricName:"areaUnderPR"}))
print("LinearRegression areaUnderROC for train dataset:", binary_evaluator.evaluate(train_pre, {binary_evaluator.metricName:"areaUnderROC"}))

print("LinearRegression areaUnderPR for test dataset:", binary_evaluator.evaluate(test_pre, {binary_evaluator.metricName:"areaUnderPR"}))
print("LinearRegression areaUnderROC for test dataset:", binary_evaluator.evaluate(test_pre, {binary_evaluator.metricName:"areaUnderROC"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LinearRegression areaUnderPR for train dataset: 0.9694515921839907
LinearRegression areaUnderROC for train dataset: 0.9084236463411847
LinearRegression areaUnderPR for test dataset: 0.9687004896898969
LinearRegression areaUnderROC for test dataset: 0.9133905313145707

### LogistRegression

In [29]:
# build logistregression model
logist_lr = LogisticRegression(labelCol="Churn", featuresCol="features")
logist_model = logist_lr.fit(train_dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1220



In [30]:
# train the dataset by using logist regression model
train_pre_logist = logist_model.transform(train_dataset)
test_pre_logist = logist_model.transform(test_dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# use the evaluator the evaluate train and test dataset
print("LogistRegression Accuracy for train dataset:", evaluator.evaluate(train_pre_logist))
print("LogistRegression Accuracy for test dataset:", evaluator.evaluate(test_pre_logist))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LogistRegression Accuracy for train dataset: 0.8532386479957974
LogistRegression Accuracy for test dataset: 0.8572162102619808

### RandomForest Regression

In [32]:
# build RandomForest model
forest = RandomForestClassifier(labelCol="Churn", featuresCol="features", maxDepth=10)
forest_model = forest.fit(train_dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# train the dataset by using Random Forest model
train_pre_forest = forest_model.transform(train_dataset)
test_pre_forest = forest_model.transform(test_dataset)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
print("RandomForest Regression Accuracy for train dataset:", evaluator.evaluate(train_pre_forest, {evaluator.metricName:"accuracy"}))
print("RandomForest Regression F1 Score for train dataset:", evaluator.evaluate(train_pre_forest, {evaluator.metricName:"f1"}))

print("RandomForest Regression Accuracy for test dataset:", evaluator.evaluate(test_pre_forest, {evaluator.metricName:"accuracy"}))
print("RandomForest Regression F1 for test dataset:", evaluator.evaluate(test_pre_forest, {evaluator.metricName:"f1"}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForest Regression Accuracy for train dataset: 0.8642411776049367
RandomForest Regression F1 Score for train dataset: 0.8670061781061109
RandomForest Regression Accuracy for test dataset: 0.8525222551928784
RandomForest Regression F1 for test dataset: 0.8514235493053522

### GridSearch

In [35]:
linearregression_dict = {
    lr.regParam: [0, 0.01, 0.1],
    lr.fitIntercept: [True, False],
    lr.maxIter: [10, 30, 50]
}
logistregression_dict = {
    logist_lr.maxIter: [10, 30, 50],
    logist_lr.fitIntercept: [True, False],
    logist_lr.regParam: [0, 0.01, 0.1]
}
forest_dict = {forest.maxDepth : [5, 10, 30], 
            forest.minInstancesPerNode : [1, 10, 30], 
            forest.maxBins : [5, 10, 30], 
            forest.numTrees: [3, 10, 30]}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:

all_module_dicts = [linearregression_dict, logistregression_dict, forest_dict]
params_maps = [reduce(lambda gb, param: gb.addGrid(*param), 
                    module.items(), 
                    ParamGridBuilder()).build() for module in all_module_dicts]

all_module_names = ["LinearRegression", "LogistRegression", "RandomForestRegression"]
all_modules = [lr, logist_lr, forest]
all_evaluates = [binary_evaluator] + [evaluator] * 2
all_module_combine = list(zip(all_module_names,all_modules, params_maps, all_evaluates))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
modules = {}
for module_name, module, param, eva in all_module_combine:
    cross = CrossValidator(estimator=module, estimatorParamMaps=param, evaluator=eva ,numFolds=3)
    cModel = cross.fit(train_dataset)
    
    train_result = cModel.transform(train_dataset)
    test_result = cModel.transform(test_dataset)
    accuracy, f1 = ("accuracy", "f1") if isinstance(eva, MulticlassClassificationEvaluator) else ("areaUnderPR", "areaUnderROC")
    print(f"{module_name} {accuracy.capitalize()} for Train dataset:", eva.evaluate(train_result, {eva.metricName: accuracy}))
    print(f"{module_name} {f1.capitalize()} for Train dataset:", eva.evaluate(train_result, {eva.metricName: f1}))

    print(f"{module_name} {accuracy.capitalize()} for Test dataset:", eva.evaluate(test_result, {eva.metricName: accuracy}))
    print(f"{module_name} {f1.capitalize()} for Test dataset:", eva.evaluate(test_result, {eva.metricName: f1}))
    
    print(f"Best params for {module_name}:", cModel.getEstimatorParamMaps()[np.argmax(cModel.avgMetrics)])
    print()
    
    modules[module_name] = {"module": cModel, "evaluator": eva}

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.9.152:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [39]:
# base on final result , use linearregression as the final model
final_module_dict = modules["LinearRegression"]

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.9.152:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [40]:
final_module = final_module_dict["module"]
val_tr = final_module.transform(val_dataset)

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.9.152:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [41]:
final_evalutor = final_module_dict["evaluator"]
print("Val dataset accuracy:", final_evalutor.evaluate(val_pre, {final_evalutor.metricName: "areaUnderPR"}))

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.9.152:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [42]:
# base on 3 model on test dataset, choose linearregression as the final model
# val_prediction = model.transform(val_dataset)
# print("Val dataset accuracy:", evaluator.evaluate(val_prediction))

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.9.152:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


###### Accuracy：准确率，该指标的高低能直接决定模型的完善程度，但需要结合训练集和测试集来下定论，如果训练集准确率太高，但训练集准确率过低，那模型就会过拟合；如果训练集和测试集的accuracy都太低，则说明欠拟合。

###### 超参数调优：
     对于随机森林模型，一开始使用的树根深度是10，其他参数默认的情况下拿捏不准具体参数，使用网格搜索后发现：maxDepth、minInstancesPerNode、maxBins、numTrees这几个超参数分别在 XX, XX, XX, XX的时候通过CrossValidator认为准确率最高；
     对于线性回归模型：regParam、fitIntercept、maxIter分别为XX，XX，XX时准确率最高；
     对于logic regression：regParam、fitIntercept、maxIter分别为XX，XX，XX时准确率最高。

###### 难点：过程中发现超参数调优是比较难的，而且由于数据集庞大，每次运行都要花飞好长时间，在AWS里面时间意味着美元；但完成了第一版项目以后觉得各个模型之间的优势劣势和侧重点依然不太清晰明了，以上三个模型都是在心里牢记的模型，至于哪一个更适用于本次项目，脑海里始终是"二元分类线性回归较好"，对概念还不是很明确，可能在以后的工作中才能慢慢熟悉。

###### 代码的改进：一开始没有使用网格搜索导致完全使用参数默认值去判定模型的优劣，这是不准确的，后来经过多次搜索后慢慢掌握了网格搜索的用法(课程里面的恕在下愚笨)，才慢慢通过网格搜索来对超参数进行调优。

